# Time-series ICESat1 & 2

1. Plotting time-series ICESat 1
2. Plotting time-series ICESat 1 & 2
3. Plotting them using box and whisker plot

## Plotting time-series ICESat 1

### Load the ICESat 1

In [ ]:
import os
from pathlib import Path
import h5py
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import time
from astropy.time import Time
import datetime
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
def glah14_to_df(filename,bounds):
    ## sp_ex = [103.643, 104.667, 12.375, 13.287]
    ## Bounds are [Longitude_West, Longitude_East, Latitude_South, Latitude_North]
    f = h5py.File(filename, 'r')
    lat = f['Data_40HZ']['Geolocation']['d_lat'][:]
    lon = f['Data_40HZ']['Geolocation']['d_lon'][:]
    elev = f['Data_40HZ']['Elevation_Surfaces']['d_elev'][:]
    sec = f['Data_40HZ']['Elevation_Corrections']['d_satElevCorr'][:]
    scf = f['Data_40HZ']['Quality']['sat_corr_flg'][:]
    satndx = f['Data_40HZ']['Quality']['i_satNdx'][:]
    dem = f['Data_40HZ']['Geophysical']['d_DEM_elv'][:]
    date_gps = f['Data_40HZ']['DS_UTCTime_40'][1] + 630763213 
    date_ymd = Time(date_gps,format='gps').datetime.strftime("%Y-%m-%d")
    
    glah14_df = pd.DataFrame({'Latitude':lat,'Longitude':lon,'Elevation':elev,
                            's_El_Corr':sec, 's_Corr_f':scf,'in_sat':satndx,
                            'DEM':dem,'Date':date_ymd})
    #### Subsetting
    glah14_df_subset = glah14_df.loc[(glah14_df['Longitude']>=bounds[0]) 
                          & (glah14_df['Longitude']<=bounds[1])
                          & (glah14_df['Latitude']>=bounds[2])
                          & (glah14_df['Latitude']<=bounds[3])]
    return glah14_df_subset

In [ ]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/icesat/')
## list them up and check them
files= list(data_home.glob('*.H5'))
### Spatial Bounds: 
tsl_sp_ex = [103.643, 104.667, 12.375, 13.287]
### load the water mask
tsl_wm = gpd.read_file('/home/jovyan/ICESat_water_level/extraction/shp/tsl_sample_dis.shp') ## water mask: Tonle Sap Lake

In [ ]:
#### convert to geoDF and clip it. 
is1_pd = pd.DataFrame()
for num in range(1,len(files)):
    gdf = gpd.GeoDataFrame(glah14_to_df(files[num],tsl_sp_ex))
    gdf.set_geometry(
        gpd.points_from_xy(gdf['Longitude'], gdf['Latitude']),
        inplace=True, crs='EPSG:4326')
    gdf_wm = gpd.clip(gdf, tsl_wm)
    is1_pd = is1_pd.append(gdf_wm)
    print(num)

### plotting one track

In [ ]:
test2 = is1_lst[0]
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test2['Date'],test2['Elevation'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface_ICESat1')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

### Removing outliers and plot it.

In [ ]:
### outliers
def out_rmv(df,var):
    ### lower (25%)
    lwq = df[var].quantile(0.25)
    ### upper (75%)
    hwq = df[var].quantile(0.75)

    ### IQR
    iqr = hwq - lwq

    lw_out = lwq-1.5*iqr
    hw_out = hwq+1.5*iqr

    ###LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR
    return df.loc[(df[var] >= lw_out) & (df[var] <= hw_out)]

In [ ]:
### I made a function for stacking the information
def gl14_a_out(db,var2):
    gl14_df = pd.DataFrame()
    date_list = db['Date'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date']==prd]
        gl14_df = gl14_df.append(out_rmv(s_db,var2), ignore_index = True)
    return gl14_df

#### Average

In [ ]:
### I made a function for averaginf the height
def gl14_mean(db,var2):
    mean_df_list = pd.DataFrame()
    date_list = db['Date'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date']==prd]
        av_height = statistics.mean(s_db[var2])
        mean_df = pd.DataFrame({'Av_level':av_height,'Date':s_db['Date'].unique()})
        mean_df_list = mean_df_list.append(mean_df, ignore_index = True)
    return mean_df_list

### Plottting time-series water level changes

In [17]:
fig=plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.set_ylim([-20,0])
ax.plot(icesat1['Date'],icesat1['Elevation'],'.', color = 'orange', markersize=0.25, label='all segements')
ax.plot(icesat1['Date'],icesat1['Elevation'],'.', color = 'green',markersize=0.4, label='Without outliers')
##ax.plot(av_h['Date'],av_h['Av_level'],'+', color = 'red',markersize=3, label='Averaged')
##ax.plot(av_h['Date'],av_h['Av_level'],color = 'red',linewidth=0.25)
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'icesat1' is not defined

## Second way to remove outliers